In [ ]:
import itertools
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing dataset is stored locally
df=pd.read_csv('/content/drive/MyDrive/Colab/mbti_1.csv', dtype={'type': str, 'posts': str})
df.info()
# Für spätere visualisierung
mbti = {'I':'Introversion', 'E':'Extroversion', 'N':'Intuition',
        'S':'Sensing', 'T':'Thinking', 'F': 'Feeling',
        'J':'Judging', 'P': 'Perceiving'}

In [ ]:
#Anzahl der verschiedenen Typen zählen
df_counted = df['type'].value_counts()
df_counted.head()

plt.figure(figsize=(12,4))
sns.barplot(x = df_counted.index, y = df_counted.values)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Types', fontsize=12)
plt.show()

# Vorverarbeitung
- ' Am anfang und Ende des Strings entfernen
- Nach ||| in eine Liste an Posts Splitten
- lowercase
- Links zählen und entfernen **IDEE:** Statt links zu entfernen diese mit einem eindeutigen Keyword ersetzen sowas wie <link>
- Abkürzungen auflösen
- Satzzeichen entfernen
- Mehrere vorkommen von einem Buchstaben in einem Wort zusammenfassen auf zwei vorkomnisse (loooove -> loove)
- Zahlen und Wörter mit Zahlen entfernen
- MBTI Kennzeichungen entfernen, da diese in diesem Datensatz sehr häufig vertreten
- Überflüssige Leerzeichen entfernt
- Leere Listenelemente entfernt, da einige Posts nur aus Links bestanten, welche gelöscht wurden

In [ ]:
def dequote(s):
    """
    If a string has single or double quotes around it, remove them.
    Make sure the pair of quotes match.
    If a matching pair of quotes is not found, return the string unchanged.
    """
    if (s[0] == s[-1]) and s.startswith(("'", '"')):
        return s[1:-1]
    return s

#Anführungszeichen entfernen vorne und hinten
df['posts_processed_dequote'] = df['posts'].apply(dequote)

In [ ]:
#Split an ||| 
df['posts_processed_dequote_split']=df['posts_processed_dequote'].apply(lambda x: x.split('|||'))
print('||| Split')

In [ ]:
#lower
df['posts_processed_dequote_split_lowercase']=df['posts_processed_dequote_split'].apply(lambda posts : list(map(str.lower, posts)))
print('lowercase')
print('Vergleich')
print(df.at[0, 'posts_processed_dequote_split'])
print(df.at[0, 'posts_processed_dequote_split_lowercase'])

In [ ]:
#Links in den Kommentaren Zählen

def count_links_in_list (list):
    link_counter = 0
    for e in list:
        link_counter += e.count('http')
    return link_counter

df['links_in_posts'] = df['posts_processed_dequote_split_lowercase'].apply(count_links_in_list)
print('Count Links')

In [ ]:
#Durchschnitt Links pro 50 Posts
df['avg_link_per_post'] = df['links_in_posts'].divide(50.0)

In [ ]:
#Durchschnitt Links pro 50 Posts visualisieren
plt.figure(figsize=(15,10))
sns.stripplot(x='type', y = 'avg_link_per_post', s = 4,data = df)

In [ ]:
# link removal
def remove_URL(stringliteral):
    """Remove URLs from a sample string"""
    return re.sub(r'https?://\S+', '', str(stringliteral))

df['posts_processed_dequote_split_lowercase_linksRemoved']=df['posts_processed_dequote_split_lowercase'].apply(lambda posts : list(map(remove_URL, posts)))
print('Removed Links')
print('Before')
print(df.at[0, 'posts_processed_dequote_split_lowercase'])
print('After')
print(df.at[0, 'posts_processed_dequote_split_lowercase_linksRemoved'])

**Achtung** : Ab hier nicht mehr immer 50 Posts pro Zeile, da einige Posts nur aus einem Link bestanden und diese nun entfernt wurden -> Leeres Element in der Liste an Posts.

In [ ]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","i've": "i have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not",
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}



# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions
#df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions']=df['posts_processed_dequote_split_lowercase_linksRemoved'].apply(lambda x : expand_contractions(x))
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions']=df['posts_processed_dequote_split_lowercase_linksRemoved'].apply(lambda posts : list(map(lambda post : expand_contractions(post), posts)))
print('Abkürzungen entfernen')
print('Before')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved'])
print('After')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions'])

In [ ]:
# Ausrufezeichen und Fragezeichen zählen, bevor sie entfernt werden

def count_punctuation_in_list (list, punct):
    counter = 0
    for e in list:
        counter += e.count(punct)
    return counter

df['exclamations_in_posts'] = df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions'].apply(lambda list : count_punctuation_in_list(list, '!'))
df['questions_in_posts'] = df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions'].apply(lambda list : count_punctuation_in_list(list, '?'))
print('Count Exclamation and Question Marks')

In [ ]:
# Benutzung von Exclamation und Question Marks visualisieren
fig, ax1 = plt.subplots(figsize=(15,10))
sns.stripplot(x='type', y = 'exclamations_in_posts', s = 4, color="#77b0ff", alpha=0.6, data = df, ax=ax1)
ax1.set_ylabel('Exclamation Marks', color="#77b0ff")
ax2 = ax1.twinx()
sns.stripplot(x='type', y = 'questions_in_posts', s = 4, color="#ff7777", alpha=0.3, data = df, ax=ax2)
ax2.set_ylabel('Question Marks', color="#ff7777")

In [ ]:
#Satzzeichen mit Leerzeichen ersetzen (muss nach den contractions passieren, da diese ebenfalls entfernt werden )
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions'].apply(lambda posts : list(map(lambda post: re.sub('[%s]' % re.escape(string.punctuation), ' ', post), posts)))
print('Satzzeichen entfernt')
print('Before')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions'])
print('After')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved'])

In [ ]:
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved'].apply(lambda posts : list(map(lambda post : ''.join(''.join(s)[:2] for _, s in itertools.groupby(post)), posts)))
print('Mehr als 2 Buchstaben hinereinander zusammengefasst')

In [ ]:
#Zahlen und Wörter mit Zahlen entfernen
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed'].apply(lambda posts : list(map(lambda post: re.sub('\w*\d\w*','', post), posts)))
print('Zahlen und Wörter mit Zahlen entfernt')
print('Before')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed'])
print('After')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers'])

In [ ]:
#MBTI Strings entfernen
mbti_strings = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
mbti_strings = [e.lower() for e in mbti_strings]

def remove_mbti_strings (post):
    for e in mbti_strings:
        post = post.replace(e,'')
    return post

df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers'].apply(lambda posts : list(map(remove_mbti_strings, posts)))
print('MBTI Strings entfernt')
print('Before')
print(df.at[3, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers'])
print('After')
print(df.at[3, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings'])

In [ ]:
# Mehrere Leerzeichen hintereinander mit einem Leerzeichen ersetzen.
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings'].apply(lambda posts : list(map(lambda post: re.sub(' +',' ',post), posts)))
print('Leerzeichen zusammengefasst')
print('Before')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings'])
print('After')
print(df.at[1, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces'])

In [ ]:
#Leere Listenelemente Löschen
df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces_removedEmptyPosts']=df['posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces'].apply(lambda posts : list(filter(None, posts)))
print('Leere Listenelemente gelöscht')
print('Before')
print(df.at[0, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces'])
print('After')
print(df.at[0, 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces_removedEmptyPosts'])

## Feature Extraktion
**Achtung** : Ab hier hat die Liste an Posts eine unterschiedliche Anzahl an Elementen

In [ ]:
#Neuen Dataframe erstellen, mit dem Weitergearbeitet werden kann
selected_columns = df[['type','exclamations_in_posts','questions_in_posts', 'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces_removedEmptyPosts','links_in_posts','avg_link_per_post']]
df_processed = selected_columns.copy()
#Umbenennen
df_processed = df_processed.rename(columns={'posts_processed_dequote_split_lowercase_linksRemoved_expandedContractions_punctuationRemoved_multipleLettersSquashed_removedNumbers_removedMbtiStrings_removedSpaces_removedEmptyPosts': 'posts'})
df_processed['post_amount'] = df_processed['posts'].apply(lambda posts : len(posts))
df_processed.head()

In [ ]:
#Wörter per Post
df_processed['words_per_post'] = df_processed['posts'].apply(lambda posts : list(map(lambda post: len(post.split()), posts)))
df_processed.head(5)

In [ ]:
# Durchschnitt der Wörter pro Post
df_processed['avg_words_of_posts'] = df_processed['words_per_post'].apply(lambda word_list : np.mean(word_list))
print('Durchschnittl. words per Post')
df_processed.head(5)

In [ ]:
plt.figure(figsize=(15,10))
sns.stripplot(x = "type", y = "avg_words_of_posts", data=df_processed)

In [ ]:
#Buchstaben pro Posts
df_processed['characters_per_post'] = df_processed['posts'].apply(lambda posts : list(map(lambda post: len(post) - post.count(' '), posts)))
print('Buchstaben pro Post berechnen')
print(df_processed.at[0,'posts'])
df_processed.head(5)

In [ ]:
# Durchschnitt der Buchstaben pro Posts
df_processed['avg_characters_of_posts'] = df_processed['characters_per_post'].apply(lambda word_list : np.mean(word_list))
print('Durchschnittl. Buchstaben per Post')
df_processed.head(5)

In [ ]:
plt.figure(figsize=(15,10))
sns.stripplot(x = "type", y = "avg_characters_of_posts", data=df_processed)

Wortwolken

In [ ]:
# Wortwolken
from functools import reduce
import wordcloud as wc

def splitAndCombine(l):
  return reduce(lambda x,y:x+y,[x.split() for x in l])

def countFrequency(l):
  freq = {}
  for word in l:
    if (word in freq):
      freq[word] += 1
    else:
      freq[word] = 1
  
  return freq

typeWithPostsDict = {k:v for k,v in zip(df['type'], df['posts_processed_dequote_split'])}

typeWithWordsDict = {t:splitAndCombine(pL) for t,pL in typeWithPostsDict.items()}

typeWithWordFrequencyDict = {t:countFrequency(wL) for t,wL in typeWithWordsDict.items()}

typeWithWordClouds = {t:wc.WordCloud(width=500, height=500, max_font_size=64).fit_words(wF) for t,wF in typeWithWordFrequencyDict.items()}

for t,wC in typeWithWordClouds.items():
  plt.figure(dpi=450)
  plt.title(t)
  plt.imshow(wC, interpolation='bilinear')
  plt.axis("off")

  plt.savefig(str(t) + ".png")
  files.download(str(t) + ".png") 

  plt.show

Binäre repräsentation der MBTI-Typen erstellen

In [ ]:
#Aufteilung in mehrere binäre Entscheidungen als bool wert.
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}
df_processed['I-E'] = df_processed['type'].astype(str).str[0]
df_processed['I-E'] = df_processed['I-E'].map(map1)
df_processed['N-S'] = df_processed['type'].astype(str).str[1]
df_processed['N-S'] = df_processed['N-S'].map(map2)
df_processed['T-F'] = df_processed['type'].astype(str).str[2]
df_processed['T-F'] = df_processed['T-F'].map(map3)
df_processed['J-P'] = df_processed['type'].astype(str).str[3]
df_processed['J-P'] = df_processed['J-P'].map(map4)
df_processed.head(5)

Da in den Wortwolen so viele stop words sind wollte ich diese auch mal entfernen


Liste von einzelnen Posts zu einer einzigen Liste mit allen Wörtern aus den Posts zusammenfügen

In [ ]:
# Function to convert
def listToString(s):

    # initialize an empty string
    str1 = " "

    # return string
    return (str1.join(s))

df_processed['posts_as_one_list'] = df_processed['posts'].apply(lambda posts : listToString(posts))
print('Posts zu einer Liste an Strings zusammenfügen')
df_processed.head(5)

In [ ]:
#Siehe https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

df_processed['posts_tokenized'] = df_processed['posts_as_one_list'].apply(word_tokenize)
print('Tokenisieren')
df_processed.head(5)

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords (tokens):
    result = []
    for word in list(tokens):
        if word not in stop_words:
            result.append(word)
    return result


df_processed['posts_without_stopwords_tokenized'] = df_processed['posts_tokenized'].apply(remove_stopwords)
print('Stop wörter entfernen')
df_processed.head(5)

Wortwolken nachdem Stopwörter entfernt wurden

In [ ]:
def splitAndCombine(l):
  return reduce(lambda x,y:x+y,[x.split() for x in l])

def countFrequency(l):
  freq = {}
  for word in l:
    if (word in freq):
      freq[word] += 1
    else:
      freq[word] = 1

  return freq

typeWithPostsDict = {k:v for k,v in zip(df_processed['type'], df_processed['posts_without_stopwords_tokenized'])}

typeWithWordsDict = {t:splitAndCombine(pL) for t,pL in typeWithPostsDict.items()}

typeWithWordFrequencyDict = {t:countFrequency(wL) for t,wL in typeWithWordsDict.items()}

typeWithWordClouds = {t:wc.WordCloud(width=500, height=500, max_font_size=64).fit_words(wF) for t,wF in typeWithWordFrequencyDict.items()}

for t,wC in typeWithWordClouds.items():
  plt.figure(dpi=150)
  plt.title(t)
  plt.imshow(wC, interpolation='bilinear')
  plt.axis("off")

  if str(t) == "ENTP" :
    plt.savefig(str(t) + ".png")
    files.download(str(t) + ".png")

  plt.show

**Da die Anzahl an Posts unterschiedlich sind müssen wir mit Durchnschnitten rechnen, daher müssen die ? und ! noch als durchschnitt berechnet werden**

In [ ]:
df_processed['avg_exclamations_in_posts'] = np.divide(df_processed['exclamations_in_posts'],df_processed['post_amount'])
df_processed['avg_questions_in_posts'] = np.divide(df_processed['questions_in_posts'],df_processed['post_amount'])
df_processed.head()

In [ ]:
value_list = [1]
df_processed['I_E'] = df_processed['I-E']
df_processed['N_S'] = df_processed['N-S']
df_processed['T_F'] = df_processed['T-F']
df_processed['J_P'] = df_processed['J-P']
#df_processed[df_processed.I_E.isin(value_list)]
df_IE = df_processed.groupby('I_E')['links_in_posts', 'exclamations_in_posts', 'questions_in_posts', 'avg_words_of_posts', 'post_amount'].agg('sum')
df_NS = df_processed.groupby('N_S')['links_in_posts', 'exclamations_in_posts', 'questions_in_posts', 'avg_words_of_posts', 'post_amount'].agg('sum')
df_TF = df_processed.groupby('T_F')['links_in_posts', 'exclamations_in_posts', 'questions_in_posts', 'avg_words_of_posts', 'post_amount'].agg('sum')
df_JP = df_processed.groupby('J_P')['links_in_posts', 'exclamations_in_posts', 'questions_in_posts', 'avg_words_of_posts', 'post_amount'].agg('sum')

df_IE['links_in_posts'] = np.divide(df_IE['links_in_posts'],df_IE['post_amount'])
df_IE['exclamations_in_posts'] = np.divide(df_IE['exclamations_in_posts'],df_IE['post_amount'])
df_IE['questions_in_posts'] = np.divide(df_IE['questions_in_posts'],df_IE['post_amount'])
df_IE['avg_words_of_posts'] = np.divide(df_IE['avg_words_of_posts'],df_IE['post_amount'])

df_NS['links_in_posts'] = np.divide(df_NS['links_in_posts'],df_NS['post_amount'])
df_NS['exclamations_in_posts'] = np.divide(df_NS['exclamations_in_posts'],df_NS['post_amount'])
df_NS['questions_in_posts'] = np.divide(df_NS['questions_in_posts'],df_NS['post_amount'])
df_NS['avg_words_of_posts'] = np.divide(df_NS['avg_words_of_posts'],df_NS['post_amount'])

df_TF['links_in_posts'] = np.divide(df_TF['links_in_posts'],df_TF['post_amount'])
df_TF['exclamations_in_posts'] = np.divide(df_TF['exclamations_in_posts'],df_TF['post_amount'])
df_TF['questions_in_posts'] = np.divide(df_TF['questions_in_posts'],df_TF['post_amount'])
df_TF['avg_words_of_posts'] = np.divide(df_TF['avg_words_of_posts'],df_TF['post_amount'])

df_JP['links_in_posts'] = np.divide(df_JP['links_in_posts'],df_JP['post_amount'])
df_JP['exclamations_in_posts'] = np.divide(df_JP['exclamations_in_posts'],df_JP['post_amount'])
df_JP['questions_in_posts'] = np.divide(df_JP['questions_in_posts'],df_JP['post_amount'])
df_JP['avg_words_of_posts'] = np.divide(df_JP['avg_words_of_posts'],df_JP['post_amount'])
df_IE

In [ ]:
df_NS

In [ ]:
df_TF

In [ ]:
df_JP

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
sns.stripplot(x=df_IE.index, y = 'links_in_posts', s = 4, color="#77b0ff", data = df_IE, ax=ax1)
ax1.set_ylabel('Average Exclamation Marks', color="#77b0ff")
ax2 = ax1.twinx()
sns.stripplot(x=df_IE.index, y = 'exclamations_in_posts', s = 4, color="#ff7777", data = df_IE, ax=ax2)
ax2.set_ylabel('Average Question Marks', color="#ff7777")

In [ ]:
# Durchschnitt von Exclamation und Question Marks visualisieren
fig, ax1 = plt.subplots(figsize=(15,10))
sns.stripplot(x='type', y = 'avg_exclamations_in_posts', s = 4, color="#77b0ff", alpha=0.6, data = df_processed, ax=ax1)
ax1.set_ylabel('Average Exclamation Marks', color="#77b0ff")
ax2 = ax1.twinx()
#sns.stripplot(x='type', y = 'avg_questions_in_posts', s = 4, color="#ff7777", alpha=0.3, data = df, ax=ax2)
ax2.set_ylabel('Average Question Marks', color="#ff7777")

Hier der Dataframe mit den ganzen Features, welche wir für die verschiedene ML ansätze benutzen können (**Ggf. ergänzen**)

In [ ]:
# Durchschnitt von Exclamation pro Ausprägung
fig, ax1 = plt.subplots(figsize=(15,10))
sns.stripplot(x='I_E', y = 'avg_exclamations_in_posts', s = 4, color="#77b0ff", alpha=0.6, data = df_processed[df_processed.I_E.isin(value_list)], ax=ax1)
ax1.set_ylabel('Average Exclamation Marks', color="#77b0ff")
ax2 = ax1.twinx()
sns.stripplot(x='I_E', y = 'avg_questions_in_posts', s = 4, color="#ff7777", alpha=0.3, data = df_processed[~df_processed.I_E.isin(value_list)], ax=ax2)
ax2.set_ylabel('Average Question Marks', color="#ff7777")

In [ ]:
#Neuen Dataframe erstellen, mit dem Weitergearbeitet werden kann
selected_columns_2 = df_processed[['type','I-E','N-S','T-F','J-P','posts_without_stopwords_tokenized','avg_exclamations_in_posts', 'avg_questions_in_posts','avg_words_of_posts','avg_link_per_post']]
df_final = selected_columns_2.copy()
df_final.head()


In [ ]:
import tensorflow
from keras.preprocessing.text import Tokenizer
num_words = 10000
oov_token = '-NA-'
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(df_final['posts_without_stopwords_tokenized'])
df_final['tokens_keras'] = tokenizer.texts_to_sequences(df_final['posts_without_stopwords_tokenized'])

# summarize what was learned
#print(tokenizer.word_counts)  #A dictionary of words and their counts.
#print(tokenizer.document_count) #An integer count of the total number of documents that were used to fit the Tokenizer.
print('Zuordnung')
print(tokenizer.word_index) #A dictionary of words and their uniquely assigned integers.
#print(tokenizer.word_docs) # A dictionary of words and how many documents each appeared in.
df_final.head()

Hier kommt der Split der Daten und anschließend das Training, aber vorher müssen diese noch normalisiert werden und richtig encoded

In [ ]:
from sklearn.model_selection import train_test_split

def one_hot_encode_sequence(seq, dim):
    result = np.zeros((len(seq), dim)) #create a all-zero matrix of shape [seq, dim]
    for row, col in enumerate(seq):
        result[row, col] = 1
    return result

# input (X) and output (y) columns
x_raw = df_final.drop(['type'], axis=1).values

data = np.array([x[9] for x in x_raw])
labels = np.array([list(x) for x in df_final['type'].values])

print(len(data))
print(len(labels))

print(labels)

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(labels)

for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))

(train_data, test_data, train_labels, test_labels) = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Train Data with {} samples. Example: {} has {} words".format(len(train_data), train_data[0], len(train_data[0])))
print("Train Data {} samples.  Label Example: {}".format(len(train_labels),train_labels[0]))
print("Test Data has {} samples".format(test_data.shape[0]))

x_train = one_hot_encode_sequence(train_data, num_words)
x_test = one_hot_encode_sequence(test_data, num_words)

print(x_train[1,:100])
print("x_train is a matrix of {}x{}".format(x_train.shape[0],x_train.shape[1]))
print("x_test is a matrix of {}x{}".format(x_test.shape[0],x_test.shape[1]))

In [ ]:
#put some - e.g. 20% - train samples as cross validation (dev) set aside
x_train_samples, x_val_samples, y_train_samples, y_val_samples = train_test_split(x_train, train_labels, train_size=0.80)

print("x_train_samples is a matrix of {}x{}".format(x_train_samples.shape[0],x_train_samples.shape[1]))
print("y_train_samples is an array of length:{}".format(y_train_samples.shape[0]))
print("x_val_samples is a matrix of {}x{}".format(x_val_samples.shape[0],x_val_samples.shape[1]))
print("y_val_samples is an array of length:{}".format(y_val_samples.shape[0]))

In [ ]:
from keras import models
from keras import utils
from keras import layers
from keras import losses
from keras import optimizers

#hyperparameter

hiddenlayer_neurons=64
hiddenlayer_activation_function='relu'

model = models.Sequential()

#1st layer - input layer
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function, input_shape=(num_words,)))
#2nd layer
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))

#...
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))
model.add(layers.Dense(units= hiddenlayer_neurons, activation=hiddenlayer_activation_function))


#last layer -output layer - has 46 outputs (len of topics array)
model.add(layers.Dense(len(mlb.classes_), activation='sigmoid'))


#An optimizer. The gradient descent algorithm - this could be the string identifier of an existing optimizer
# such as rmsprop or adagrad
# see also: https://keras.io/optimizers/

#loss: The loss function - the objective that the model will try to minimize. 
#It can be the string identifierof an existing loss function (such as categorical_crossentropy or mse)


model.compile(optimizer=optimizers.RMSprop(learning_rate=0.0001),
             loss = losses.binary_crossentropy, #.mean_squared_error, #categorical_crossentropy
             metrics=['accuracy'])

In [ ]:
history = model.fit (
                    x_train_samples,
                    y_train_samples,
                    epochs = 30, # how many time to repeat the learning process for the entire dataset
                    batch_size=128, # we will reset the weights batch-size wise (mini batch GD)
                    validation_data = (x_val_samples, y_val_samples)
                    )

In [ ]:
result = model.evaluate(x_test,test_labels) # final evaluation of the model
print("result on the testset: accuracy={}".format(result[1]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

history_dict = history.history
#print(history_dict)

amount_epochs = len(history_dict['accuracy']) 
plt.plot(range(1, amount_epochs+1), history_dict['loss'], 'ro', label='Training Set Loss')
plt.plot(range(1, amount_epochs+1), history_dict['val_loss'], 'r', label='Validation Set Loss')

plt.xlabel('Training Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.plot(range(1, amount_epochs+1), history_dict['accuracy'], 'bo', label='Training Set Accuracy')
plt.plot(range(1, amount_epochs+1), history_dict['val_accuracy'], 'b', label='Validation Set Accuracy')

plt.xlabel('Training Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(x_test)
print("predictions is a matrix of {}x{}".format(predictions.shape[0],predictions.shape[1]))

In [ ]:
index = 1734
print("prediction: \n {} \n\n"
      .format(predictions[index]
      ))

print("ground truth label class: {}".format(test_labels[index]))

result = []

im_classlist =  [x for x in predictions[index]]
m_classList = [x for x in predictions[index]]

for i in range(4) :
  print(m_classList)
  maxItem = max(m_classList)
  result.append(im_classlist.index(maxItem))
  m_classList.remove(maxItem)

print("found indices: " + str(result))

print("class in binarizer: " + str(mlb.classes_))

# I E S N F T J P
letters = [0, 2, 6, 4, 1, 7, 3, 5]

result.sort(key = lambda i: letters.index(i))

print("MBTI: {}".format([mlb.classes_[x] for x in result]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = df_processed['posts_without_stopwords_tokenized'].apply(lambda x : ' '.join(x))

print(sentences)

tfid = TfidfVectorizer(ngram_range=(2,4), min_df=0.005, sublinear_tf=False)

from sklearn.preprocessing import MultiLabelBinarizer

labels = np.array([list(x) for x in df_processed['type'].values])

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split

(train_data, test_data, train_labels, test_labels) = train_test_split(tfid.fit_transform(df_final['sentences'].reset_index(drop=True)).sorted_indices(), labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)

x_train_samples, x_val_samples, y_train_samples, y_val_samples = train_test_split(train_data, train_labels, train_size=0.80)

print(x_train_samples.shape)
print(x_val_samples.shape)

print(x_train_samples[0])
print(y_train_samples[0])

In [ ]:
print(x_train_samples[1])
print(y_train_samples[1])

In [ ]:
model_tdif = models.Sequential()
model_tdif.add(layers.Dense(units=64 , activation='relu', input_shape=(7082,)))
model_tdif.add(layers.Dense(units=64 , activation='relu'))
model_tdif.add(layers.Dense(len(mlb.classes_), activation='sigmoid'))

model_tdif.compile(optimizer=optimizers.RMSprop(learning_rate=0.00001),
             loss = losses.categorical_crossentropy,
             metrics=['accuracy'])

In [ ]:
history = model_tdif.fit (
                    x_train_samples,
                    y_train_samples,
                    epochs = 30,
                    batch_size=128,
                    verbose=1,
                    validation_data = (x_val_samples, y_val_samples)
                    )

In [ ]:
result = model_tdif.evaluate(test_data,test_labels) # final evaluation of the model
print("result on the testset: accuracy={}".format(result[1]))

In [ ]:
predictions = model_tdif.predict(test_data)
print("predictions is a matrix of {}x{}".format(predictions.shape[0],predictions.shape[1]))

In [ ]:
index = 1734
print("prediction: \n {} \n\n"
      .format(predictions[index]
      ))

print("ground truth label class: {}".format(test_labels[index]))

result = []

im_classlist =  [x for x in predictions[index]]
m_classList = [x for x in predictions[index]]

for i in range(4) :
  print(m_classList)
  maxItem = max(m_classList)
  result.append(im_classlist.index(maxItem))
  m_classList.remove(maxItem)

print("found indices: " + str(result))

print("class in binarizer: " + str(mlb.classes_))

# I E S N F T J P
letters = [0, 2, 6, 4, 1, 7, 3, 5]

result.sort(key = lambda i: letters.index(i))

print("MBTI: {}".format([mlb.classes_[x] for x in result]))

# **Multi-Class Text Classification**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

sentences = df_processed['posts_without_stopwords_tokenized'].apply(lambda x : ' '.join(x))

print(sentences)
print(df_processed['type'])

X = sentences
y = df_processed['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

In [ ]:
nb_gram = Pipeline([('vect', CountVectorizer(ngram_range=(2,3))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb_gram.fit((X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb_gram.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_gram = Pipeline([('vect', CountVectorizer(ngram_range=(2,3))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

sgd_gram.fit(X_train, y_train)

y_pred = sgd_gram.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, max_iter=150)),
               ])

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_gram = Pipeline([('vect', CountVectorizer(ngram_range=(2,3))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, max_iter=100)),
               ])

logreg_gram.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=types))

https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

# **Multi-Label Classifier**

Run **ONE** of the vectorizers based on preference:

In [ ]:
feature_count = 50000

***NGram (1, 1)***

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), lowercase=False,max_features=feature_count)

***NGram (1, 2)***

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), lowercase=False ,max_features=feature_count)

***NGram (2, 2)***

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(2,2), lowercase=False ,max_features=feature_count)

**NGram (1, 3)**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), lowercase=False, max_features=feature_count)

**NGram (2, 3)**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(2,3), lowercase=False, max_features=feature_count)

**NGram (3, 3)**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(3,3), lowercase=False, max_features=feature_count)

**Binary Classes**

In [ ]:
df_split_types = df_processed['type'].apply(lambda x : [c for c in x])

flippedClasses =    ['E','S','T','J']
unflippedClasses =  ['I','N','F','P']

df_binary_classes = df_split_types.apply(lambda x : [1 if c in flippedClasses else 0 for c in x])

print(df_binary_classes.values)

binary_class_arr =  np.asarray(df_binary_classes.values.tolist(), np.int64)

print(binary_class_arr)

Fitting and Transforming

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

y = binary_class_arr

vectorizer.fit(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
row = X_train_tfidf.getrow(0)
feature_count = len(vectorizer.vocabulary_)

sum = 0.0

for y in range(size):
  row = X_train_tfidf.getrow(y)
  row_sum = 0.0
  for x in range(feature_count):
    row_sum += row[0,x]
  row_sum /= feature_count
  sum += row_sum
  print(sum)

print(sum / size)

In [ ]:
def value_sum (values):
  sum = 0.0
  for value in values:
    sum += value
  return sum

vocab_dict = vectorizer.vocabulary_

length = len(vocab_dict)
sum = value_sum(vectorizer.vocabulary_.values())

print(sum / len)

Visualation of Frequencies

In [ ]:
from yellowbrick.text import FreqDistVisualizer
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features, orient='v', n=75, size=(1080, 540))
visualizer.fit(X_train_tfidf)

In [ ]:
features = vectorizer.get_feature_names()

td_idf_m0 = X_train_tfidf.getrow(0)

size = td_idf_m0.shape[1]

td_idf_arr0 = []

td_idf_arr0 = [td_idf_m0[0,i] for i in range(size)]

td_idf_dict0 = {features[i]: td_idf_arr0[i] for i in range(size) if td_idf_arr0[i] >= 0.02}

plt.figure(dpi=300)

plt.bar(*zip(*td_idf_dict0.items()))
plt.xticks(rotation=90)

ax = plt.gca()
ax.tick_params(axis = 'both', which = 'major', labelsize = 12)
ax.tick_params(axis = 'both', which = 'minor', labelsize = 8)

plt.tight_layout()

#plt.savefig("td_idf_0.png")
#files.download("td_idf_0.png") 

plt.show()

**Models**

https://www.geeksforgeeks.org/an-introduction-to-multilabel-classification/

https://medium.com/technovators/machine-learning-based-multi-label-text-classification-9a0e17f88bb4

In [ ]:
!pip install scikit-multilearn

from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score, f1_score
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.neural_network import MLPClassifier

NB

In [ ]:
nbClassifier = OneVsRestClassifier(MultinomialNB())
nbClassifier.fit(X_train_tfidf, y_train)

nbPreds = nbClassifier.predict(X_test_tfidf)

print(accuracy_score(y_test, nbPreds))
print(hamming_loss(y_test, nbPreds))
print(f1_score(y_test, nbPreds, average='macro'))

Multi-Label K-Nearest-Neighbour

In [ ]:
mlknn_classifier = MLkNN()
mlknn_classifier.fit(X_train_tfidf, y_train)

mlknnPreds = mlknn_classifier.predict(X_test_tfidf)
  
print(accuracy_score(y_test, mlknnPreds))
print(hamming_loss(y_test, mlknnPreds))
print(f1_score(y_test, mlknnPreds, average='macro'))

Support Vector Machine

In [ ]:
svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(X_train_tfidf, y_train)

svmPreds = svmClassifier.predict(X_test_tfidf)

print(accuracy_score(y_test, svmPreds))
print(hamming_loss(y_test, svmPreds))
print(f1_score(y_test, svmPreds, average='macro'))

Binary Relevance

In [ ]:
binaryRelSVC = BinaryRelevance(LinearSVC())
binaryRelSVC.fit(X_train_tfidf, y_train)

binaryRelSVCPreds = binaryRelSVC.predict(X_test_tfidf)

print(accuracy_score(y_test, binaryRelSVCPreds))
print(hamming_loss(y_test, binaryRelSVCPreds))
print(f1_score(y_test, binaryRelSVCPreds, average='macro'))

Label Powerset

In [ ]:
powerSetSVC = LabelPowerset(LinearSVC())
powerSetSVC.fit(X_train_tfidf, y_train)

powerSetSVCPreds = powerSetSVC.predict(X_test_tfidf)

print(accuracy_score(y_test, powerSetSVCPreds))
print(hamming_loss(y_test, powerSetSVCPreds))
print(f1_score(y_test, powerSetSVCPreds, average='macro'))

Multi-layer Perceptron classifier

In [ ]:
mlp = MLPClassifier(random_state=42, max_iter=50, hidden_layer_sizes=64, verbose=True)
mlp.fit(X_train_tfidf, y_train)

print(mlp.score(X_test_tfidf, y_test))

In [ ]:
print(mlp.loss_curve_)

Visualization of specfic prediction

In [ ]:
index = 254

def binary_to_mbti_str(binary_arr):
  return "".join([flippedClasses[i] if x == 1 else unflippedClasses[i] for i, x in enumerate(binary_arr)])

print("prediction: \n {} \n"
      .format(powerSetSVCPreds[index]
      ))

print("ground truth label class:\t{}".format(y_test[index]))
print("ground truth MBTI: \t\t{}".format(binary_to_mbti_str(y_test[index])))

setClasses = powerSetSVCPreds.rows[index]

result = [0] * 4

for c in setClasses:
  result[c] = 1

print("\nfound classes:\t\t\t{}".format(np.array(result)))
print("found MBTI: \t\t\t{}".format(binary_to_mbti_str(np.array(result))))